In [1]:
import pandas as pd 
import numpy as np

In [3]:
pwd

'/Users/lisa/Documents/Uni/ML4RG/linkpred'

In [26]:
chia_pet = pd.read_csv("NBFNet/data/ext_val/external_val_chia-pet.tsv", sep="\t")
crispri = pd.read_csv("NBFNet/data/ext_val/external_val_crispri.tsv", sep="\t")
train = pd.read_csv("NBFNet/data/lnctard/train.txt", sep="\t", names=["head", "relation", "tail"])
val = pd.read_csv("/Users/lisa/Documents/Uni/ML4RG/linkpred/NBFNet/data/lnctard/valid.txt", sep="\t", names=["head", "relation", "tail"])
train_val = pd.concat([train, val], axis = 0, ignore_index=True)

In [10]:
val.head()

,head,relation,tail
0,LEF1-AS1,expression association,LEF1
1,ZEB1-AS1,expression association,BAX
2,HNF1A-AS1,transcriptional regulation,OTX1
3,SCARNA2,ceRNA or sponge,EGFR
4,LINC00473,expression association,MMP9


In [11]:
train.head()

,head,relation,tail
0,91H,epigenetic regulation,H19
1,91H,epigenetic regulation,IGF2
2,91H,interact with protein,HNRNPK
3,A2M-AS1,expression association,CD2
4,A2M-AS1,expression association,IL1R2


In [29]:
train_val["key"] = train_val[["head", "tail"]].apply(lambda row: tuple(sorted(row)), axis=1)
chia_pet["key"] = chia_pet[["head", "tail"]].apply(lambda row: tuple(sorted(row)), axis=1)
crispri["key"] = crispri[["head", "tail"]].apply(lambda row: tuple(sorted(row)), axis=1)
train_val.set_index("key")
chia_pet.set_index("key")
crispri.set_index("key")

,head,relation,tail
key,,,
"(MARCKS, PVT1_2)",PVT1_2,affects,MARCKS
"(MIR29A, NAGK)",MIR29A,affects,NAGK
"(E2F1, PVT1_1)",PVT1_1,affects,E2F1
"(SNHG12, TTC37)",SNHG12,affects,TTC37
"(CTPS1, RP11-126L15.4)",RP11-126L15.4,affects,CTPS1
...,...,...,...
"(MIR29A, WRB)",MIR29A,affects,WRB
"(CCAT1_2, EDIL3)",CCAT1_2,affects,EDIL3
"(CCAT1_2, SAT1)",CCAT1_2,affects,SAT1


In [16]:
train_val.head()

,head,relation,tail,key
0,91H,epigenetic regulation,H19,"(91H, H19)"
1,91H,epigenetic regulation,IGF2,"(91H, IGF2)"
2,91H,interact with protein,HNRNPK,"(91H, HNRNPK)"
3,A2M-AS1,expression association,CD2,"(A2M-AS1, CD2)"
4,A2M-AS1,expression association,IL1R2,"(A2M-AS1, IL1R2)"


Exclude triplets from external test set that are present in validation or training set

In [52]:
value = train_val["key"].tolist()
chia_filtered = chia_pet.loc[~chia_pet['key'].isin(value)]

In [51]:
mask = ~crispri["key"].isin(train_val["key"])
crispri_filtered = crispri.loc[mask]

In [54]:
print(f"Overlap Chia Pet: {len(chia_pet)-len(chia_filtered)}")
print(f"Overlap Crispri: {len(crispri)-len(crispri_filtered)}")

Overlap Chia Pet: 3
Overlap Crispri: 12


Exclude triplets that contain nodes that were not present in the training data

In [61]:
train_nodes = pd.concat([train["head"], train["tail"]]).unique()

In [57]:
len(list_of_nodes)

2963

In [71]:
mask_nodes_crisp = (crispri_filtered["head"].isin(train_nodes) & crispri_filtered["tail"].isin(train_nodes))
mask_nodes_chia = (chia_filtered["head"].isin(train_nodes) & chia_filtered["tail"].isin(train_nodes))

In [74]:
len(mask_nodes_chia)

2657

In [75]:
len(chia_filtered)

2657

In [76]:
chia_filtered= chia_filtered.loc[mask_nodes_crisp]
crispri_filtered = crispri_filtered.loc[mask_nodes_chia]

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).